# Indexación jerárquica

Hasta este punto nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas `Series` y `DataFrame`, respectivamente.
A menudo resulta útil ir más allá y almacenar datos de dimensiones superiores, es decir, datos indexados por más de una o dos claves.
Las primeras versiones de Pandas proporcionaban objetos `Panel` y `Panel4D` que podían considerarse análogos 3D o 4D del `DataFrame` 2D, pero eran algo complicados de usar en la práctica. Un patrón mucho más común para manejar datos de dimensiones superiores es hacer uso de *indexación jerárquica* (también conocida como *indexación múltiple*) para incorporar múltiples *niveles* de índice dentro de un único índice.
De esta manera, los datos de dimensiones superiores se pueden representar de forma compacta dentro de los familiares objetos unidimensionales "Series" y bidimensionales "DataFrame".
(Si está interesado en matrices verdaderas *N*-dimensionales con índices flexibles estilo Pandas, puede consultar el excelente [paquete Xarray](https://xarray.pydata.org/).)

En este capítulo, exploraremos la creación directa de objetos `MultiIndex`; consideraciones al indexar, dividir y calcular estadísticas a través de datos indexados múltiples; y rutinas útiles para convertir entre representaciones de datos simples y indexadas jerárquicamente.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

## Una serie indexada multiplicada

Comencemos considerando cómo podríamos representar datos bidimensionales dentro de una "Serie" unidimensional.
Para mayor concreción, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### El mal camino

Suponga que desea realizar un seguimiento de los datos sobre estados de dos años diferentes.
Usando las herramientas de Pandas que ya hemos cubierto, es posible que tengas la tentación de usar simplemente tuplas de Python como claves:

In [2]:
index = [('California', 2010), ('California', 2020),
         ('New York', 2010), ('New York', 2020),
         ('Texas', 2010), ('Texas', 2020)]
populations = [37253956, 39538223,
               19378102, 20201249,
               25145561, 29145505]
pop = pd.Series(populations, index=index)
pop

(California, 2010)    37253956
(California, 2020)    39538223
(New York, 2010)      19378102
(New York, 2020)      20201249
(Texas, 2010)         25145561
(Texas, 2020)         29145505
dtype: int64

Con este esquema de indexación, puede indexar o dividir la serie de manera sencilla según este índice de tupla:

In [3]:
pop[('California', 2020):('Texas', 2010)]

(California, 2020)    39538223
(New York, 2010)      19378102
(New York, 2020)      20201249
(Texas, 2010)         25145561
dtype: int64

Pero la comodidad termina ahí. Por ejemplo, si necesita seleccionar todos los valores de 2010, necesitará realizar algunas modificaciones complicadas (y potencialmente lentas) para que esto suceda:

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Esto produce el resultado deseado, pero no es tan limpio (ni tan eficiente para grandes conjuntos de datos) como la sintaxis de corte que nos encanta en Pandas.

### La mejor manera: Pandas MultiIndex
Afortunadamente, Pandas ofrece una mejor manera.
Nuestra indexación basada en tuplas es esencialmente un multiíndice rudimentario, y el tipo `MultiIndex` de Pandas nos proporciona los tipos de operaciones que deseamos tener.
Podemos crear un índice múltiple a partir de las tuplas de la siguiente manera:

In [5]:
index = pd.MultiIndex.from_tuples(index)

El `MultiIndex` representa múltiples *niveles* de indexación (en este caso, los nombres de los estados y los años), así como múltiples *etiquetas* para cada punto de datos que codifica estos niveles.

Si reindexamos nuestra serie con este `MultiIndex`, vemos la representación jerárquica de los datos:

In [6]:
pop = pop.reindex(index)
pop

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

Aquí, las dos primeras columnas de la representación de la Serie muestran los múltiples valores del índice, mientras que la tercera columna muestra los datos.
Observe que faltan algunas entradas en la primera columna: en esta representación de índices múltiples, cualquier entrada en blanco indica el mismo valor que la línea que está encima.

Ahora, para acceder a todos los datos cuyo segundo índice es 2020, podemos usar la notación de división de Pandas:

In [7]:
pop[:, 2020]

California    39538223
New York      20201249
Texas         29145505
dtype: int64

El resultado es una serie indexada individualmente con solo las claves que nos interesan.
Esta sintaxis es mucho más conveniente (¡y la operación es mucho más eficiente!) que la solución de indexación múltiple basada en tuplas casera con la que comenzamos.
Ahora analizaremos más a fondo este tipo de operación de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión adicional

Quizás notes algo más aquí: podríamos haber almacenado fácilmente los mismos datos usando un simple `DataFrame` con etiquetas de índice y columna.
De hecho, Pandas se creó teniendo en cuenta esta equivalencia. El método `unstack` convertirá rápidamente una `Series` indexada múltiples veces en un `DataFrame` indexado convencionalmente:

In [8]:
pop_df = pop.unstack()
pop_df

,2010,2020
California,37253956,39538223
New York,19378102,20201249
Texas,25145561,29145505


Naturalmente, el método ``stack`` proporciona la operación opuesta:

In [9]:
pop_df.stack()

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

Al ver esto, quizás se pregunte por qué nos molestaríamos en absoluto con la indexación jerárquica.
La razón es simple: así como pudimos usar la indexación múltiple para manipular datos bidimensionales dentro de una "Serie" unidimensional, también podemos usarlo para manipular datos de tres o más dimensiones en una "Serie" o " Marco de datos`.
Cada nivel adicional en un índice múltiple representa una dimensión adicional de datos; Aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer agregar otra columna de datos demográficos para cada estado cada año (digamos, población menor de 18 años); con un ``MultiIndex` esto es tan fácil como agregar otra columna al ``DataFrame``:

In [10]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9284094, 8898092,
                                   4318033, 4181528,
                                   6879014, 7432474]})
pop_df

total  under18
California 2010  37253956  9284094
           2020  39538223  8898092
New York   2010  19378102  4318033
           2020  20201249  4181528
Texas      2010  25145561  6879014
           2020  29145505  7432474

Además, todas las ufuncs y otras funciones analizadas en [Operación de datos en Pandas] (03.03-Operaciones-en-Pandas.ipynb) también funcionan con índices jerárquicos.
Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2010,2020
California,0.249211,0.225050
New York,0.222831,0.206994
Texas,0.273568,0.255013


Esto nos permite manipular y explorar fácil y rápidamente incluso datos de alta dimensión.

## Métodos de creación de múltiples índices

La forma más sencilla de construir una "Serie" o un "Marco de datos" con índices múltiples es simplemente pasar una lista de dos o más matrices de índice al constructor. Por ejemplo:

In [12]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.748464  0.561409
  2  0.379199  0.622461
b 1  0.701679  0.687932
  2  0.436200  0.950664

El trabajo de creación del ``MultiIndex`` se realiza en segundo plano.

De manera similar, si pasa un diccionario con tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y usará un ``MultiIndex`` de forma predeterminada:

In [13]:
data = {('California', 2010): 37253956,
        ('California', 2020): 39538223,
        ('New York', 2010): 19378102,
        ('New York', 2020): 20201249,
        ('Texas', 2010): 25145561,
        ('Texas', 2020): 29145505}
pd.Series(data)

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

Sin embargo, a veces resulta útil crear explícitamente un "MultiIndex"; A continuación veremos un par de métodos para hacer esto.

### Constructores multiíndice explícitos

Para obtener más flexibilidad en cómo se construye el índice, puede utilizar los métodos constructores disponibles en la clase `pd.MultiIndex`.
Por ejemplo, como hicimos antes, puedes construir un `MultiIndex` a partir de una lista simple de matrices que proporcionen los valores de índice dentro de cada nivel:

In [14]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

O puedes construirlo a partir de una lista de tuplas que proporcionen los múltiples valores de índice de cada punto:

In [15]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Incluso puedes construirlo a partir de un producto cartesiano de índices únicos:

In [16]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

De manera similar, puedes construir un `MultiIndex` directamente usando su codificación interna pasando `levels` (una lista de listas que contienen valores de índice disponibles para cada nivel) y `codes` (una lista de listas que hacen referencia a estas etiquetas):

In [17]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Cualquiera de estos objetos puede pasarse como argumento `index` al crear una `Series` o `DataFrame`, o pasarse al método `reindex` de una `Series` o `DataFrame` existente.

### Nombres de niveles de índice múltiple

A veces conviene nombrar los niveles del `MultiIndex`.
Esto se puede lograr pasando el argumento `names` a cualquiera de los constructores `MultiIndex` discutidos anteriormente, o estableciendo el atributo `names` del índice después del hecho:

In [18]:
pop.index.names = ['state', 'year']
pop

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

Con conjuntos de datos más complicados, esta puede ser una forma útil de realizar un seguimiento del significado de varios valores de índice.

### Índice múltiple para columnas

En un "DataFrame", las filas y columnas son completamente simétricas y, así como las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles.
Considere lo siguiente, que es una maqueta de algunos datos médicos (algo realistas):

In [19]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      30.0  38.0  56.0  38.3  45.0  35.8
     2      47.0  37.1  27.0  36.0  37.0  36.4
2014 1      51.0  35.9  24.0  36.7  32.0  36.2
     2      49.0  36.3  48.0  39.2  31.0  35.7

Se trata de datos fundamentalmente de cuatro dimensiones, donde las dimensiones son el tema, el tipo de medición, el año y el número de visita.
Con esto implementado, podemos, por ejemplo, indexar la columna de nivel superior por el nombre de la persona y obtener un `DataFrame` completo que contenga solo la información de esa persona:

In [20]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      56.0  38.3
     2      27.0  36.0
2014 1      24.0  36.7
     2      48.0  39.2

## Indexación y división de un MultiIndex

La indexación y el corte en un "MultiIndex" están diseñados para ser intuitivos y resulta útil pensar en los índices como dimensiones adicionales.
Primero veremos la indexación de objetos "Series" con índice múltiple y luego objetos "DataFrame" con índice múltiple.

### Multiplicar series indexadas

Considere la "Serie" de poblaciones estatales con índices múltiples que vimos anteriormente:

In [21]:
pop

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

Podemos acceder a elementos individuales indexando con múltiples términos:

In [22]:
pop['California', 2010]

37253956

El `MultiIndex` también admite *indexación parcial*, o indexación solo de uno de los niveles del índice.
El resultado es otra `Serie`, manteniéndose los índices de nivel inferior:

In [23]:
pop['California']

year
2010    37253956
2020    39538223
dtype: int64

El corte parcial también está disponible, siempre y cuando el `MultiIndex` esté ordenado (consulte la discusión en [Índices ordenados y sin clasificar](#Índices-ordenados-y-sin-clasificar)):

In [24]:
pop.loc['California':'New York']

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
dtype: int64

Con índices ordenados, se puede realizar una indexación parcial en niveles inferiores pasando un segmento vacío en el primer índice:

In [25]:
pop[:, 2010]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

Otros tipos de indexación y selección (que se analizan en [Indexación y selección de datos](03.02-Indexación-y-Selección-de-datos.ipynb)) también funcionan; por ejemplo, selección basada en máscaras booleanas:

In [26]:
pop[pop > 22000000]

state       year
California  2010    37253956
            2020    39538223
Texas       2010    25145561
            2020    29145505
dtype: int64

La selección basada en una indexación elegante también funciona:

In [27]:
pop[['California', 'Texas']]

state       year
California  2010    37253956
            2020    39538223
Texas       2010    25145561
            2020    29145505
dtype: int64

### Multiplicar marcos de datos indexados

Un `DataFrame` indexado varias veces se comporta de manera similar.
Considere nuestro juguete médico `DataFrame` de antes:

In [28]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      30.0  38.0  56.0  38.3  45.0  35.8
     2      47.0  37.1  27.0  36.0  37.0  36.4
2014 1      51.0  35.9  24.0  36.7  32.0  36.2
     2      49.0  36.3  48.0  39.2  31.0  35.7

Recuerde que las columnas son primarias en un `DataFrame`, y la sintaxis utilizada para las `Series` indexadas múltiples se aplica a las columnas.
Por ejemplo, podemos recuperar los datos de frecuencia cardíaca de Guido con una sencilla operación:

In [29]:
health_data['Guido', 'HR']

year  visit
2013  1        56.0
      2        27.0
2014  1        24.0
      2        48.0
Name: (Guido, HR), dtype: float64

Además, al igual que en el caso de índice único, podemos usar los indexadores `loc`, `iloc` y `ix` introducidos en [Indexación y selección de datos] (03.02-Indexación-y-selección-de-datos.ipynb). Por ejemplo:

In [30]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      30.0  38.0
     2      47.0  37.1

Estos indexadores proporcionan una vista similar a una matriz de los datos bidimensionales subyacentes, pero a cada índice individual en `loc` o `iloc` se le puede pasar una tupla de múltiples índices. Por ejemplo:

In [31]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        30.0
      2        47.0
2014  1        51.0
      2        49.0
Name: (Bob, HR), dtype: float64

Trabajar con sectores dentro de estas tuplas de índice no es especialmente conveniente; intentar crear un segmento dentro de una tupla generará un error de sintaxis:

In [32]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (3311942670.py, line 1)

Podrías solucionar esto construyendo el segmento deseado explícitamente usando la función `slice` incorporada de Python, pero una mejor manera en este contexto es usar un objeto `IndexSlice`, que Pandas proporciona precisamente para esta situación.
Por ejemplo:

In [33]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,30.0,56.0,45.0
2014,1,51.0,24.0,32.0


Como puede ver, hay muchas maneras de interactuar con datos en ``Series` y ``DataFrame``s indexados múltiples y, como ocurre con muchas de las herramientas de este libro, la mejor manera de familiarizarse con ellas es probarlas.

## Reorganización de índices múltiples

Una de las claves para trabajar con datos indexados múltiples es saber cómo transformar los datos de manera efectiva.
Hay una serie de operaciones que conservarán toda la información del conjunto de datos, pero la reorganizarán para realizar diversos cálculos.
Vimos un breve ejemplo de esto en los métodos "apilar" y "desapilar", pero hay muchas más formas de controlar con precisión la reorganización de datos entre índices y columnas jerárquicos, y las exploraremos aquí.

### Índices ordenados y sin clasificar

Anteriormente mencioné brevemente una advertencia, pero debería enfatizarla más aquí.
*Muchas de las operaciones de división `MultiIndex` fallarán si el índice no está ordenado.*
Echemos un vistazo más de cerca.

Comenzaremos creando algunos datos indexados múltiples simples donde los índices *no están ordenados lexográficamente*:

In [34]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.280341
      2      0.097290
c     1      0.206217
      2      0.431771
b     1      0.100183
      2      0.015851
dtype: float64

Si intentamos tomar una porción parcial de este índice, se producirá un error:

In [35]:
try:
    data['a':'b']
except KeyError as e:
    print("KeyError", e)

KeyError 'Key length (1) was greater than MultiIndex lexsort depth (0)'


Aunque no queda del todo claro en el mensaje de error, este es el resultado de que el "MultiIndex" no está ordenado.
Por varias razones, los cortes parciales y otras operaciones similares requieren que los niveles en el "MultiIndex" estén ordenados (es decir, lexográficos).
Pandas proporciona una serie de rutinas convenientes para realizar este tipo de clasificación, como los métodos `sort_index` y `sortlevel` del `DataFrame`.
Usaremos el más simple, `sort_index`, aquí:

In [36]:
data = data.sort_index()
data

char  int
a     1      0.280341
      2      0.097290
b     1      0.100183
      2      0.015851
c     1      0.206217
      2      0.431771
dtype: float64

Con el índice ordenado de esta manera, el corte parcial funcionará como se esperaba:

In [37]:
data['a':'b']

char  int
a     1      0.280341
      2      0.097290
b     1      0.100183
      2      0.015851
dtype: float64

### Apilar y desapilar índices

Como vimos brevemente antes, es posible convertir un conjunto de datos de un índice múltiple apilado a una representación bidimensional simple, especificando opcionalmente el nivel a usar:

In [38]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2010,37253956,19378102,25145561
2020,39538223,20201249,29145505


In [39]:
pop.unstack(level=1)

year,2010,2020
state,,
California,37253956,39538223
New York,19378102,20201249
Texas,25145561,29145505


Lo opuesto a "desapilar" es "apilar", que aquí se puede utilizar para recuperar la serie original:

In [40]:
pop.unstack().stack()

state       year
California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

### Configuración y restablecimiento del índice

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de índice en columnas; Esto se puede lograr con el método `reset_index`.
Llamar a esto en el diccionario de población dará como resultado un "DataFrame" con columnas "estado" y "año" que contienen la información que anteriormente estaba en el índice.
Para mayor claridad, opcionalmente podemos especificar el nombre de los datos para la representación de la columna:

In [41]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2010,37253956
1,California,2020,39538223
2,New York,2010,19378102
3,New York,2020,20201249
4,Texas,2010,25145561
5,Texas,2020,29145505


Un patrón común es crear un "MultiIndex" a partir de los valores de las columnas.
Esto se puede hacer con el método `set_index` del `DataFrame`, que devuelve un `DataFrame` indexado varias veces:

In [42]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2010    37253956
           2020    39538223
New York   2010    19378102
           2020    20201249
Texas      2010    25145561
           2020    29145505

En la práctica, este tipo de reindexación es uno de los patrones más útiles al explorar conjuntos de datos del mundo real.